<a href="https://colab.research.google.com/github/Mtiwari27/Virality-Prediction-Model/blob/main/Viralityapp_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pickle
import re
import pandas as pd
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load the trained model
with open("virality_model.pkl", "rb") as file:
    model = pickle.load(file)

# Function to preprocess text
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # Remove URLs
    text = re.sub(r'\@\w+|\#','', text)  # Remove mentions & hashtags
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.lower()  # Convert to lowercase
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])  # Remove stopwords
    return text

# Function to extract features
def extract_features(post_text, likes, shares, comments, followers, post_hour):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(post_text)['compound']
    hashtag_count = len(re.findall(r"#\w+", post_text))
    engagement_rate = (likes + shares + comments) / max(followers, 1)  # Avoid division by zero

    time_of_day = [0, 0, 0, 0]  # [Night, Morning, Afternoon, Evening]
    if 0 <= post_hour < 6:
        time_of_day[0] = 1
    elif 6 <= post_hour < 12:
        time_of_day[1] = 1
    elif 12 <= post_hour < 18:
        time_of_day[2] = 1
    else:
        time_of_day[3] = 1

    return [hashtag_count, sentiment, engagement_rate] + time_of_day

# Streamlit UI
st.title("Social Media Virality Predictor")
st.write("Enter the details of your post to predict if it will go viral.")

# User input
post_text = st.text_area("Enter your post text:")
likes = st.number_input("Number of Likes:", min_value=0, value=100)
shares = st.number_input("Number of Shares:", min_value=0, value=50)
comments = st.number_input("Number of Comments:", min_value=0, value=20)
followers = st.number_input("Total Followers:", min_value=1, value=1000)
post_hour = st.slider("Hour of Post (0-23):", min_value=0, max_value=23, value=12)

if st.button("Predict"):
    features = extract_features(post_text, likes, shares, comments, followers, post_hour)
    prediction = model.predict([features])[0]

    if prediction == 1:
        st.success("This post is likely to go VIRAL! 🚀")
    else:
        st.warning("This post may not go viral. Try optimizing hashtags and engagement.")